In [1]:
!pip install ultralytics
# ==============================
# STEP 1: Install Required Libraries
# ==============================
import os
import torch
from google.colab import drive
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:

# ==============================
# STEP 2: Mount Google Drive
# ==============================
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = "/content/drive/MyDrive/Dental Dataset"

In [4]:
# ==============================
# STEP 4: Verify Dataset Structure
# ==============================
assert os.path.exists(f"{dataset_dir}/train/images"), "❌ Train images folder not found!"
assert os.path.exists(f"{dataset_dir}/valid/images"), "❌ Validation images folder not found!"
assert os.path.exists(f"{dataset_dir}/test/images"), "❌ Test images folder not found!"
assert os.path.exists(f"{dataset_dir}/data.yaml"), "❌ data.yaml not found!"

print("✅ Dataset structure is correct!")

✅ Dataset structure is correct!


In [5]:
# ==============================
# STEP 5: Check Device
# ==============================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [6]:
# ==============================
# STEP 6: Load Pretrained YOLOv8 Model
# ==============================
model = YOLO("yolov8m.pt")   # using medium version

In [7]:
# ==============================
# STEP 7: Train the Model
# ==============================
model.train(
    data=f"{dataset_dir}/data.yaml",  # dataset yaml path
    epochs=40,          # increase if needed
    imgsz=1024,         # larger size for small speed boards
    batch=8,            # adjust to GPU RAM
    workers=4,
    optimizer="Adam",
    device=device
)

print("✅ Training complete!")

Ultralytics 8.3.198 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Dental Dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, pl

In [12]:
 # ==============================
# STEP 8: Save Trained Model to Drive
# ==============================
trained_model_path = "runs/detect/train/weights/best.pt"
drive_save_path = "/content/drive/MyDrive/YOLOv8_SpeedLimit/best.pt"

os.makedirs(os.path.dirname(drive_save_path), exist_ok=True)

if os.path.exists(trained_model_path):
    !cp {trained_model_path} {drive_save_path}
    print(f"✅ Model saved to Google Drive: {drive_save_path}")
else:
    print("❌ Error: Trained model not found!")

✅ Model saved to Google Drive: /content/drive/MyDrive/YOLOv8_SpeedLimit/best.pt


In [10]:
# ==============================
# STEP 9: Test the Fine-Tuned Model
# ==============================
# Reload trained weights
model = YOLO(drive_save_path)

# Run inference on test images
results = model.predict(
    source=f"{dataset_dir}/test/images",
    conf=0.25,      # confidence threshold
    imgsz=1024,
    save=True       # saves annotated results
)

print("✅ Testing complete! Check 'runs/detect/predict/' for annotated results.")



image 1/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig100_SAU_JPG.rf.79b3fc155a06f078d19333b496071469.jpg: 544x1024 4 1st Molars, 4 1st Premolars, 1 2nd Molar, 4 2nd Premolars, 4 Canines, 4 Central Incisors, 4 Lateral Incisors, 49.3ms
image 2/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig105_MYS_JPG.rf.6cfdf193fc8c49102807258ffabe0969.jpg: 480x1024 4 1st Molars, 4 1st Premolars, 1 2nd Molar, 4 2nd Premolars, 4 Canines, 4 Central Incisors, 4 Lateral Incisors, 65.3ms
image 3/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig105_SAU_JPG.rf.ffabf11509d576acc8a655d6889749ad.jpg: 576x1024 3 1st Premolars, 1 2nd Premolar, 4 Canines, 4 Central Incisors, 5 Lateral Incisors, 57.6ms
image 4/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig106_SAU_JPG.rf.90850296b125ca3699dcc13d5878bad1.jpg: 512x1024 5 1st Premolars, 1 2nd Premolar, 4 Canines, 4 Central Incisors, 4 Lateral Incisors, 49.6ms
image 5/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig108_PL

In [11]:
# ==============================
# STEP 10: Test the Model Again
# ==============================
test_results = model.predict(source=f"{dataset_dir}/test/images", save=True)
print("✅ Model tested on test dataset!")


image 1/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig100_SAU_JPG.rf.79b3fc155a06f078d19333b496071469.jpg: 544x1024 4 1st Molars, 4 1st Premolars, 1 2nd Molar, 4 2nd Premolars, 4 Canines, 4 Central Incisors, 4 Lateral Incisors, 46.0ms
image 2/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig105_MYS_JPG.rf.6cfdf193fc8c49102807258ffabe0969.jpg: 480x1024 4 1st Molars, 4 1st Premolars, 1 2nd Molar, 4 2nd Premolars, 4 Canines, 4 Central Incisors, 4 Lateral Incisors, 40.2ms
image 3/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig105_SAU_JPG.rf.ffabf11509d576acc8a655d6889749ad.jpg: 576x1024 3 1st Premolars, 1 2nd Premolar, 4 Canines, 4 Central Incisors, 5 Lateral Incisors, 46.6ms
image 4/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig106_SAU_JPG.rf.90850296b125ca3699dcc13d5878bad1.jpg: 512x1024 5 1st Premolars, 1 2nd Premolar, 4 Canines, 4 Central Incisors, 4 Lateral Incisors, 41.3ms
image 5/107 /content/drive/MyDrive/Dental Dataset/test/images/Fig108_PL